## PHEME Dataset (330 Tweets)

In [ ]:
def get_src_text(row):
    path = "data/en/" + row['event'] + "/" + str(row['threadid']) + "/source-tweets/" + str(row['tweetid']) + ".json"
    with open(path, "r") as f:
        source = json.loads(f.read())
        return source['text']

def is_true(row):
    print(row)
    path = "data/en/" + row['event'] + "/" + str(row['threadid']) + "/annotation.json"
    with open(path, "r") as f:
        source = json.loads(f.read())
        return str(source.get('true', 'unverified'))

In [ ]:
# Dataset used: https://figshare.com/articles/dataset/PHEME_rumour_scheme_dataset_journalism_use_case/2068650
df = pd.read_json("data/en-scheme-annotations.json", dtype = {"threadid": str, "tweetid": str}, lines=True)

df['true'] = df.apply(lambda row: is_true(row), axis=1)
df['src_text'] = df.apply(lambda row: get_src_text(row), axis=1)

print(df.head())

In [ ]:
def get_src_text_by_index(row):
    return df.iloc[row.index]['src_text']

accuracy = pd.DataFrame([test_labels, lrtfidf_test])
print(accuracy)
print(lrtfidf_test)
print(test_labels)
print(type(test_labels))
test_df = pd.DataFrame(test_labels)
test_df['predicted'] = lrtfidf_test
print(test_df.apply(lambda row: get_src_text_by_index(row)))
print(test_df)
#test_df = pd.DataFrame({"tweet_id":test_labels[0], "actual_label": test_labels[1]})
test_labels.add(lrtfidf_test)
print(test_labels)

## Experimenting with sample of whole data

In [ ]:
shorter = train_features[:50, :50]
shorter_labels = train_labels[:50]
print(train_features[100, 666])
#print(train_features)
print(train_features.shape)
print(shorter.shape)

# SVC
svc = SVC(kernel='rbf')
svc_model = svc.fit(shorter, shorter_labels)
svc_test = svc_model.predict(test_features)
evaluation_summary("SVC test", test_labels, svc_test)

# Logistic Regression with TF-IDF
lr_tfidf = LogisticRegression()
lr_model_tfidf = lr_tfidf.fit(shorter, shorter_labels)
lrtfidf_test = lr_model_tfidf.predict(test_features)
evaluation_summary("LR (TF-IDF) test", test_labels, lrtfidf_test)

# Dummy Majority
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(shorter, shorter_labels)
dumb_test = dumb.predict(test_features)
evaluation_summary("Dummy MF test", test_labels, dumb_test)

## Emoji tokenisation

In [ ]:
nlp_spacymoji = spacy.load("en_core_web_sm")
emoji = Emoji(nlp_spacymoji, merge_spans=True)
nlp_spacymoji.add_pipe('emoji', first=True)
# tokenised_train = train_frame.apply(lambda row: nlp_spacymoji(row['src_text']))
# #tok_train = nlp_spacymoji(train_text)
# print(tokenised_train)

def spacy_tokeniser(text):
    tokens = []
    for w in nlp_spacymoji(text):
        tokens.append(w.lemma_.lower())
    return tokens

## Train-Test Splitting

In [ ]:
def split_dataset(text_col, label_col):
  train_text, temp_text, train_labels, temp_labels = train_test_split(df[text_col], df[label_col], 
                                                                      random_state=2018, 
                                                                      test_size=0.3)


  val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                  random_state=2018, 
                                                                  test_size=0.5, 
                                                                  stratify=temp_labels)

  # train_frame = pd.DataFrame([train_text, train_labels])
  # val_frame = pd.DataFrame([val_text, val_labels])
  # test_frame = pd.DataFrame([test_text, test_labels])
  return train_text, train_labels, val_text, val_labels, test_text, test_labels

train_text, train_labels, val_text, val_labels, test_text, test_labels = split_dataset('text', 'true')

print(type(train_text))
print(train_labels.value_counts())

## Addition of article (incorrect)

In [ ]:
# Experimentation - not needed
prev_article = ""
article_counter = 0
for row_ix, row in real.iterrows():
    if row['article_id'] != prev_article:
        print(f"New article at row index {row_ix}!")
        article_counter += 1
        print("Articles so far:", article_counter)
        prev_article = row['article_id']

## Time Feature Engineering

In [ ]:
import datetime
import pytz

def remove_date(row):
    #obj = datetime.strptime(row['created_at'], )
    return row['created_at'].timetz()

def get_hour(row):
    return row['created_at'].hour

def get_tz(row):
    time = row['created_at'].split('T')[1]
    time_split = re.split(r'[:|.]', time)
    return time_split[-1]

df = X.copy()
df['created_at'] = pd.to_datetime(df['created_at']) # str to datetime
# TODO: Put into section when importing dataset
df['created_at'] = df.apply(lambda row: row['created_at'].hour, axis=1)

#df.head()
df['created_at'].value_counts()

import re

def stringify_time(row):
    time = row['created_at'].split('T')[1]
    time_split = re.split(r'[:|.]', time)
    return " ".join(time_split)

df = X.copy()
df['created_at'] = df.apply(lambda row: stringify_time(row), axis=1)
df.head()

import pytz

df = X
#print(df)
df['created_at'] = df.apply(lambda row: row['created_at'].astimezone('Europe/Riga'), axis=1)
df.head()

df.iloc[0]['created_at']

## Cleaning database

In [ ]:
df = clean_dataset(X)
df['text'] = df.text.apply(nltk_preprocess)
df.head()

## Multi-Layer Perceptron

In [ ]:
# https://github.com/georgian-io/Multimodal-Toolkit/blob/master/multimodal_transformers/model/layer_utils.py

import math
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss


class MLP(nn.Module):
    """mlp can specify number of hidden layers and hidden layer channels"""

    def __init__(self, input_dim, output_dim, act='relu', num_hidden_lyr=2,
                 dropout_prob=0.5, return_layer_outs=False,
                 hidden_channels=None, bn=False):
        super().__init__()
        self.out_dim = output_dim
        self.dropout = nn.Dropout(dropout_prob)
        self.return_layer_outs = return_layer_outs
        if not hidden_channels:
            hidden_channels = [input_dim for _ in range(num_hidden_lyr)]
        elif len(hidden_channels) != num_hidden_lyr:
            raise ValueError(
                "number of hidden layers should be the same as the lengh of hidden_channels")
        self.layer_channels = [input_dim] + hidden_channels + [output_dim]
        self.act_name = act
        self.activation = create_act(act)
        self.layers = nn.ModuleList(list(
            map(self.weight_init, [nn.Linear(self.layer_channels[i], self.layer_channels[i + 1])
                                   for i in range(len(self.layer_channels) - 2)])))
        final_layer = nn.Linear(self.layer_channels[-2], self.layer_channels[-1])
        self.weight_init(final_layer,   activation='linear')
        self.layers.append(final_layer)

        self.bn = bn
        if self.bn:
            self.bn = nn.ModuleList([torch.nn.BatchNorm1d(dim) for dim in self.layer_channels[1:-1]])

    def weight_init(self, m, activation=None):
        if activation is None:
            activation = self.act_name
        torch.nn.init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain(activation))
        return m

    def forward(self, x):
        """
        :param x: the input features
        :return: tuple containing output of MLP,
                and list of inputs and outputs at every layer
        """
        layer_inputs = [x]
        for i, layer in enumerate(self.layers):
            input = layer_inputs[-1]
            if layer == self.layers[-1]:
                layer_inputs.append(layer(input))
            else:
                if self.bn:
                    output = self.activation(self.bn[i](layer(input)))
                else:
                    output = self.activation(layer(input))
                layer_inputs.append(self.dropout(output))

        # model.store_layer_output(self, layer_inputs[-1])
        if self.return_layer_outs:
            return layer_inputs[-1], layer_inputs
        else:
            return layer_inputs[-1]

## Concatenate Features (Multimodal Approach)

In [ ]:
#print(pipe['encoder'].get_feature_names_out())

# Define categorical columns
categorical = ['verified', 'created_at']
print(f"Categorical columns are: {categorical}")

lexical = ['description', 'location', 'name', 'source', 'text', 'username']
print(f"Lexical columns are: {lexical}")

df = X.copy()
for col_name in categorical:
    df[col_name] = df[col_name].astype('category')
    df[col_name] = df[col_name].cat.codes.astype('float')
    
df[categorical].head()

In [ ]:
text_combined = []
for (i, row) in df.iterrows():
    combined = ' [SEP] '.join([str(row[text_col]) for text_col in lexical])
    text_combined.append(combined)
    
df.insert(0, 'all_text', text_combined)
df.head()

In [ ]:
#pickle.dump(tfidf, open("tfidf_vectorizer.sav", "wb"))
#print(train_text)
# Fit pipeline to training data

# https://towardsdatascience.com/pipeline-columntransformer-and-featureunion-explained-f5491f815f
# Interesting read for time-related feature engineering: 
#   https://scikit-learn.org/stable/auto_examples/applications/plot_cyclical_feature_engineering.html

# class TextProcessor(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.text_transformer = Pipeline([
#             ('tfidf_vectoriser', TfidfVectorizer()),
#         ])
       
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         return self.text_transformer.fit_transform(X.squeeze()).toarray()

# class CategoryProcessor(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.text_transformer = Pipeline([
#             ('1hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)),
#         ])
       
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X, y=None):
#         return self.text_transformer.fit_transform(X.squeeze()).toarray()

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    
def get_transformers(keys):
    transformers = []
    for key in keys:
        transformers.append(
            (key, Pipeline([
                ('selector', ItemSelector(key=key)),
                ('tfidf', TfidfVectorizer())
            ]))
        )
    return transformers
    
cat_pipe = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

lex_pipe = Pipeline([
    #("squeeze", FunctionTransformer(lambda x: x.squeeze())),
    ('encoder', TfidfVectorizer()),
    #("toarray", FunctionTransformer(lambda x: x.toarray()))
])

lex_pipe = FeatureUnion([
        ('text', Pipeline([
            ('selector', ItemSelector(key='text')),
            ('tfidf', TfidfVectorizer())])),
#         ('desc', Pipeline([
#             ('selector', ItemSelector(key='description')),
#             ('tfidf', TfidfVectorizer())])),
#         ('loc', Pipeline([
#             ('selector', ItemSelector(key='location')),
#             ('tfidf', TfidfVectorizer())])),
#         ('name', Pipeline([
#             ('selector', ItemSelector(key='name')),
#             ('tfidf', TfidfVectorizer())])),
#         ('source', Pipeline([
#             ('selector', ItemSelector(key='source')),
#             ('tfidf', TfidfVectorizer())])),
#         ('user', Pipeline([
#             ('selector', ItemSelector(key='username')),
#             ('tfidf', TfidfVectorizer())])),
])

preprocessor = ColumnTransformer([
    ('lex', lex_pipe, lexical),
    ('cat', cat_pipe, categorical),
])

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(warm_start=True))
    #('model', SGDClassifier(loss='log_loss'))
])

print(X_train.shape)

mini_model = SGDClassifier(loss='log_loss')

batch_size = 1000
for batch_ix in range(0, len(X_train), batch_size):
    ix = min(batch_ix + batch_size, len(X_train))
    #pipe.fit(X_train.iloc[batch_ix:ix, :], y_train.iloc[batch_ix:ix])
    mini_model.partial_fit(X_train.iloc[batch_ix:ix, :], y_train.iloc[batch_ix:ix])

#pipe.fit(X_train, y_train) # hahaha, naive thinking this would work

# Predict test data
#y_test_pred = pipe.predict(X_test)
y_test_pred = mini_model.predict(X_test)


evaluation_summary("LR multi-feature test", y_test, t_test_pred)

In [ ]:
# https://www.youtube.com/watch?v=NbbsVcs42jE

from torch import nn
from transformers import BertForSequenceClassification

class BertConcatFeatures(BertForSequenceClassification):
    
    def __init__(self, config):
        super().__init__(config) # BERT setup
        
        self.num_labels = config.num_labels
        
        combined_feat_dim = config.text_feat_dim + \
                            config.cat_feat_dim + \
                            config.numerical_feat_dim
        
        self.num_bn = nn.BatchNorm1d(config.numerical_feat_dim)
        
        dims = []
        dim = combined_feat_dim
        while True:
            dim = dim // 4
            if dim <= self.num_labels:
                break 
            dim.append(int(dim))
        print('MLP layer sizes:')
        print('   Input:', combined_feat_dim)
        print('   Hidden:', dims)
        print('   Output:', self.num_labels)
        print('')
        
        self.mlp = MLP(combined_feat_dim,
                       self.num_labels,
                       num_hidden_layers=len(dims),
                       dropout_prob=0.1,
                       hidden_channels=dims,
                       bn=True)
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                position_ids=None, head_mask=None, inputs_embeds=None, labels=None,
                class_weights=None, output_attentions=None, output_hidden_states=None,
                cat_feats=None, numerical_feats=None):
        
        outputs = self.bert(input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds,
                            output_attentions=output_attentions,
                            output_hidden_states=output_hidden_states
                           )
        
        # outputs[0] - ALL output embeddings from BERT
        # outputs[1] - [CLS] token embedding, with additional "pooling" done
        cls = outputs[1]
        cls = self.dropout(cls)
        
        # Concatenate features
        
        numerical_feats = self.num_bn(numerical_feats) # batch-normalise numerical features
        combined_feats = torch.cat((cls, cat_feats, numerical_feats), dim=1)
        
        # Output classifier / Regression
        
        logits = self.mlp(combined_feats)
        
        if type(logits) is tuple: # regression
            logits, classifier_layer_outputs = logits[0], logits[1]
        else: # simple classifier
            classifier_layer_outputs = [combined_feats, logits]
            
        # Loss
        if labels is not None:
            # Regression
            if self.num_labels == 1:
                loss_fct = MSELoss()
                labels = labels.float()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            # Classification
            else:
                loss_fct = CrossEntropyLoss(weight=class_weights)
                labels = labels.long()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
                
        else:
            loss = None
            
        results = {
            'loss': loss,
            'logits': logits,
            'classifier_layer_outputs': classifier_layer_outputs
        }
        
        return results

## Old BERT Attempt

In [ ]:
# https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb#scrollTo=URn-DWJt5xhP
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

import transformers as ppb
model_class, tokenizer_class, bert_model_name = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(bert_model_name)
model = model_class.from_pretrained(bert_model_name)

tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

padded_encoding = df['text'].apply(
    lambda src_text: tokenizer.encode_plus(
        src_text, add_special_tokens = True,
        truncation = True, padding = "max_length", 
        return_attention_mask = True, return_tensors = "pt")
)

#input_ids = torch.tensor(np.array(padded_encoding))

# print(padded_encoding.shape)

# #new_tensor = torch.tensor([200, padded_encoding])

new_tensor = padded_encoding[None, :]
#print(new_tensor)
new_tensor[0] = 200
#print(new_tensor.shape)

with torch.no_grad():
    last_hidden_states = model(new_tensor) # ERROR: Tries to get 255 GB of RAM ============================================
    # The above line is the one responsible for abandoning this approach because this BERT implementation could not be 
    # applied in batches
    
    
features = last_hidden_states[0][:,0,:].numpy()

labels = df['true']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

lr_clf.score(test_features, test_labels) # 0.5866666666666667 score for PHEME dataset